In [ ]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np

from dynamicslib.consts import muEM
from dynamicslib.continuation import find_bif, arclen_cont
from dynamicslib.common import jacobi_constant, eom, prop
from dynamicslib.targeter import dc_arclen, dc_square, dc_overconstrained, dc_arclen
from dynamicslib.integrate import dop853
from tqdm.auto import tqdm
from dynamicslib.plotting import plotly_display, bifurcation_search
from dynamicslib.common_targetters import propagate_X, spatial_perpendicular, spatial_period_fixed, fullstate_minus_one, axial

family = "L1 Lyapunov"

root = os.getcwd()
file = f"{root}/database/{family}.csv"
db = pd.read_csv(file).set_index("Index")

In [ ]:
bifurcation_search(db, line_color_by="Jacobi Constant")

In [ ]:
# targetter = fullstate_minus_one(1, 0, 0.0, 1e-10, muEM)
targetter = axial(1e-11, muEM)

func = targetter.f_df_stm

In [ ]:
keys = ["Initial x", "Initial y", "Initial z", "Initial vx", "Initial vy", "Initial vz"]
keys += ["Period"]
for key in keys:
    if key not in db.columns:
        db[key] = 0.0

im1 = 25
im0 = 26

In [ ]:
Xm0 = targetter.get_X(db[keys[:-1]].iloc[im0].values, db["Period"].iloc[im0])
Xm1 = targetter.get_X(db[keys[:-1]].iloc[im1].values, db["Period"].iloc[im1])
Xm0, _, _ = dc_overconstrained(Xm0, func, 1e-11, debug=True)
Xm1, _, _ = dc_overconstrained(Xm1, func, 1e-11, debug=True)
f, df, stm = func(Xm0)
svd = np.linalg.svd(df)
print(f"Last dist is {np.linalg.norm(Xm0-Xm1):.3e}")

In [ ]:
X0, tan1 = find_bif(
    Xm0,
    func,
    Xm0 - Xm1,
    3e-3,
    targ_tol=1e-9,
    bisect_tol=1e-8,
    bif_type="tangent",
    debug=True,
    scale=2,
)

In [ ]:
# switch direction!
X0r, _, _ = dc_overconstrained(propagate_X(targetter, X0), func, 1e-10, debug=True)

In [ ]:
# targetter = axial(1e-10, muEM)
# func = targetter.f_df_stm
# X0t = X0r[[0, 3, 4, 5]].copy()
# X0t[-1] /= 2

X1 = X0.copy()
_, df, _ = func(X1)
tangent = np.linalg.svd(df).Vh[-2]

In [ ]:
Xsp1, esp1 = arclen_cont(
    X1, func, -tangent, 1e-3, 0.1, 1e-11, exact_tangent=True, max_iter=600
)

In [ ]:
Xsp2, esp2 = arclen_cont(
    Xsp1[-1], func, Xsp1[-1] - Xsp1[-2], 2e-3, 0.5, 1e-11, max_iter=250
)

In [ ]:
Xsp3, esp3 = arclen_cont(
    Xsp2[-1], func, Xsp2[-1] - Xsp2[-2], 3e-3, 0.25, 1e-9, max_iter=250
)

In [ ]:
Xsm1, esm1 = arclen_cont(
    X1, func, -tangent, 7e-2, 5.0, 1e-9, exact_tangent=False, max_iter=200
)

In [ ]:
Xsm2, esm2 = arclen_cont(
    Xsm1[-1], func, Xsm1[-1] - Xsm1[-2], 5e-3, 1.5, 1e-9, max_iter=250
)

In [ ]:
Xsp = [*Xsp1, *Xsp2[1:], *Xsp3[1:]]
esp = [*esp1, *esp2[1:], *esp3[1:]]
Xsm = [Xsm1[1:], Xsm2[2:]]
esm = [Xsm1[1:], Xsm2[2:]]

param_names = [
    "Index",
    "Initial x",
    "Initial y",
    "Initial z",
    "Initial vx",
    "Initial vy",
    "Initial vz",
    "Period",
    "Jacobi Constant",
    "Stability Index",
    "Eig1",
    "Eig2",
    "Eig3",
    "Eig4",
    "Eig5",
    "Eig6",
]

xyzs = []
data = []


iterator = list(enumerate(zip(Xsm, esm)))[::-1]
for i, (X, evals) in tqdm(iterator):
    Xcp = X.copy()
    x0 = targetter.get_x0(Xcp)
    tf = targetter.get_period(Xcp)
    xyzs.append(prop(x0, tf, muEM, 1e-10))
    args = np.argsort(np.abs(evals))
    stabind = np.max([(np.abs(lam) + 1 / np.abs(lam)) / 2 for lam in evals])
    jc = jacobi_constant(x0)
    x0 = targetter.get_x0(Xcp)
    data.append([-i - 1, *x0, tf, jc, stabind, *evals])

iterator = list(enumerate(zip(Xsp, esp)))
for i, (X, evals) in tqdm(iterator):
    Xcp = X.copy()
    x0 = targetter.get_x0(Xcp)
    tf = targetter.get_period(Xcp)
    xyzs.append(prop(x0, tf, muEM, 1e-9))
    args = np.argsort(np.abs(evals))
    stabind = np.max([(np.abs(lam) + 1 / np.abs(lam)) / 2 for lam in evals])
    jc = jacobi_constant(x0)
    x0 = targetter.get_x0(Xcp)
    data.append([i, *x0, tf, jc, stabind, *evals])

In [ ]:
df = pd.DataFrame(data, columns=param_names).set_index("Index")
for key in param_names[1:]:
    if np.all(df[key].abs() < 1e-8):
        if key != "Initial z":
            df = df.drop(columns=[key])
        else:
            df[key] = 0.0

df.to_csv("L1 Lyapunov-DPO Axial.csv")

In [ ]:
a = -710
b = -700
c = 1
# colscale = [
#     [0.0, "rgb(150, 0, 0)"],
#     [0.67, "rgb(0, 180, 0)"],
#     [0.9, "rgb(0, 50, 0)"],
#     [1.0, "rgb(200, 200, 200)"],
# ]
plotly_display(xyzs[a:b:c], df[a:b:c], color_by="Index")